Import Biblioteci

In [2]:
import numpy as np
import os
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import glob
import cv2 as cv
import pdb
import pickle
import ntpath
from copy import deepcopy
import timeit
from skimage.feature import hog

In [23]:
def creeaza_foldere_suport(director_output, clase):
    """
    Creează folderele pentru fiecare clasă și aspect (patrat, dreptunghi vertical, dreptunghi orizontal).
    """
    for clasa in clase:
        os.makedirs(os.path.join(director_output, clasa), exist_ok=True)




def decupeaza_fete(director_input, fisier_adnotari, director_output):
    """
    Decupează fețele din imagini conform adnotărilor și le salvează în foldere separate.
    """
    clase = ["dexter", "deedee", "mom", "dad", "unknown"]
    creeaza_foldere_suport(director_output, clase)

    with open(fisier_adnotari, 'r') as f:
        adnotari = f.readlines()

    for linie in adnotari:
        # Extrage detaliile din linia de adnotare
        nume_imagine, xmin, ymin, xmax, ymax, clasa = linie.strip().split()
        xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])

        # Încarcă imaginea corespunzătoare
        cale_imagine = os.path.join(director_input, nume_imagine)
        if not os.path.exists(cale_imagine):
            print(f"Imaginea {cale_imagine} nu există. Continuăm.")
            continue

        imagine = cv.imread(cale_imagine)
        if imagine is None:
            print(f"Nu s-a putut încărca imaginea {cale_imagine}.")
            continue

        # Decupează fața conform coordonatelor
        fata = imagine[ymin:ymax, xmin:xmax]

        # Clasifică aspectul decupajului
        latime = xmax - xmin
        inaltime = ymax - ymin


        # Creează calea unde va fi salvată fața
        cale_salveaza = os.path.join(director_output, clasa)
        nume_fisier_output = f"{nume_imagine}_{nume_imagine.split('.')[0]}_{xmin}_{ymin}_{xmax}_{ymax}.jpg"
        cv.imwrite(os.path.join(cale_salveaza, nume_fisier_output), fata)

        print(f"Salvat: {os.path.join(cale_salveaza, nume_fisier_output)}")

# Exemplu de utilizare
director_input = "antrenare/dexter"  # Înlocuiește cu calea către imaginile tale
fisier_adnotari = "antrenare/dexter_annotations.txt"  # Înlocuiește cu calea către fișierul de adnotări
director_output = "teste_pozitive"  # Înlocuiește cu calea unde vrei să salvezi datele

decupeaza_fete(director_input, fisier_adnotari, director_output)


Salvat: teste_pozitive\dexter\0001.jpg_0001_191_196_233_224.jpg
Salvat: teste_pozitive\unknown\0001.jpg_0001_77_176_127_215.jpg
Salvat: teste_pozitive\unknown\0001.jpg_0001_293_153_323_209.jpg
Salvat: teste_pozitive\unknown\0001.jpg_0001_362_190_426_226.jpg
Salvat: teste_pozitive\dexter\0002.jpg_0002_192_159_302_251.jpg
Salvat: teste_pozitive\unknown\0003.jpg_0003_41_125_210_262.jpg
Salvat: teste_pozitive\dexter\0003.jpg_0003_341_190_436_259.jpg
Salvat: teste_pozitive\dexter\0004.jpg_0004_151_89_367_223.jpg
Salvat: teste_pozitive\dexter\0005.jpg_0005_185_179_324_248.jpg
Salvat: teste_pozitive\dexter\0006.jpg_0006_10_93_131_190.jpg
Salvat: teste_pozitive\dexter\0007.jpg_0007_231_149_287_183.jpg
Salvat: teste_pozitive\dexter\0008.jpg_0008_182_101_292_200.jpg
Salvat: teste_pozitive\dexter\0009.jpg_0009_187_129_420_302.jpg
Salvat: teste_pozitive\unknown\0009.jpg_0009_3_65_155_277.jpg
Salvat: teste_pozitive\dexter\0010.jpg_0010_82_55_401_278.jpg
Salvat: teste_pozitive\dexter\0011.jpg_0011_1

In [24]:
import os
import cv2

def redimensioneaza_la_dimensiune_fixa(imagine, dimensiune):
    """
    Redimensionează imaginea la o dimensiune fixă (dimensiune x dimensiune).
    """
    imagine_redimensionata = cv2.resize(imagine, (dimensiune, dimensiune))
    return imagine_redimensionata

def proceseaza_imagini(director_input, director_output, dimensiune=256):
    """
    Parcurge imaginile din structura simplificată `teste_pozitive/[nume_pers]`,
    le redimensionează la dimensiunea fixă și le salvează într-un director de output.
    """
    for persoana in os.listdir(director_input):  # Parcurgem directoarele de personaje
        cale_persoana = os.path.join(director_input, persoana)
        if not os.path.isdir(cale_persoana):
            continue

        # Cream structura directorului de output
        cale_output = os.path.join(director_output, persoana)
        os.makedirs(cale_output, exist_ok=True)

        # Procesăm fiecare imagine din directorul curent
        for fisier in os.listdir(cale_persoana):
            if fisier.endswith(('.jpg', '.png')):  # Doar imagini
                cale_imagine = os.path.join(cale_persoana, fisier)
                imagine = cv2.imread(cale_imagine)

                if imagine is None:
                    print(f"Nu s-a putut încărca imaginea {cale_imagine}.")
                    continue

                # Redimensionăm imaginea la dimensiunea fixă
                imagine_redimensionata = redimensioneaza_la_dimensiune_fixa(imagine, dimensiune)

                # Salvează imaginea în directorul de output
                cale_salveaza = os.path.join(cale_output, fisier)
                cv2.imwrite(cale_salveaza, imagine_redimensionata)
                print(f"Salvat: {cale_salveaza}")

# Exemplu de utilizare
director_input = "teste_pozitive"  # Înlocuiește cu calea către imaginile tale
director_output = "teste_pozitive_redimensionate"  # Înlocuiește cu calea unde vrei să salvezi imaginile redimensionate

proceseaza_imagini(director_input, director_output, dimensiune=50)


Salvat: teste_pozitive_redimensionate\dad\0001.jpg_0001_242_28_429_256.jpg
Salvat: teste_pozitive_redimensionate\dad\0002.jpg_0002_143_39_257_185.jpg
Salvat: teste_pozitive_redimensionate\dad\0003.jpg_0003_349_38_403_125.jpg
Salvat: teste_pozitive_redimensionate\dad\0004.jpg_0004_173_64_356_337.jpg
Salvat: teste_pozitive_redimensionate\dad\0005.jpg_0005_330_25_432_145.jpg
Salvat: teste_pozitive_redimensionate\dad\0006.jpg_0006_176_59_293_193.jpg
Salvat: teste_pozitive_redimensionate\dad\0006.jpg_0006_64_167_137_254.jpg
Salvat: teste_pozitive_redimensionate\dad\0007.jpg_0007_138_71_261_202.jpg
Salvat: teste_pozitive_redimensionate\dad\0008.jpg_0008_105_43_214_172.jpg
Salvat: teste_pozitive_redimensionate\dad\0008.jpg_0008_71_100_172_192.jpg
Salvat: teste_pozitive_redimensionate\dad\0009.jpg_0009_43_246_109_286.jpg
Salvat: teste_pozitive_redimensionate\dad\0010.jpg_0010_105_140_317_358.jpg
Salvat: teste_pozitive_redimensionate\dad\0011.jpg_0011_187_66_293_180.jpg
Salvat: teste_pozitive_r

In [25]:
import os
import cv2
import random

def verifica_intersectie(x, y, latime, inaltime, adnotari):
    """
    Verifică dacă o fereastră definită de (x, y, latime, inaltime) se intersectează cu o zonă adnotată.
    """
    for _, xmin, ymin, xmax, ymax, _ in adnotari:
        # Verifică suprapunerea
        if not (x + latime <= xmin or x >= xmax or y + inaltime <= ymin or y >= ymax):
            return True  # Se intersectează
    return False  # Nu se intersectează

def genereaza_poze_negative(director_input, fisiere_adnotari, director_output, dim_fereastra=(64, 64), nr_neg=5):
    """
    Creează decupaje negative din imagini care nu se intersectează cu zonele adnotate.
    """
    os.makedirs(director_output, exist_ok=True)  # Creează directorul pentru imaginile negative

    # Parcurge fiecare fișier de adnotări
    for fisier_adnotari in fisiere_adnotari:
        with open(fisier_adnotari, 'r') as f:
            adnotari = f.readlines()

        # Prelucrează adnotările în format structurat
        adnotari_structurate = []
        for linie in adnotari:
            nume_imagine, xmin, ymin, xmax, ymax, clasa = linie.strip().split()
            adnotari_structurate.append((nume_imagine, int(xmin), int(ymin), int(xmax), int(ymax), clasa))

        # Parcurge imaginile menționate în fișierul de adnotări
        imagini_procesate = set()
        for nume_imagine, _, _, _, _, _ in adnotari_structurate:
            if nume_imagine in imagini_procesate:
                continue
            imagini_procesate.add(nume_imagine)

            # Încarcă imaginea
            cale_imagine = os.path.join(director_input, nume_imagine)
            imagine = cv2.imread(cale_imagine)
            if imagine is None:
                print(f"Nu s-a putut încărca imaginea {cale_imagine}.")
                continue

            inaltime, latime = imagine.shape[:2]

            # Extrage adnotările pentru imaginea curentă
            adnotari_imagine = [a for a in adnotari_structurate if a[0] == nume_imagine]

            # Generează decupaje negative
            for _ in range(nr_neg):
                x = random.randint(0, latime - dim_fereastra[0])
                y = random.randint(0, inaltime - dim_fereastra[1])

                # Verifică dacă decupajul este valid (nu se intersectează cu adnotările)
                while verifica_intersectie(x, y, dim_fereastra[0], dim_fereastra[1], adnotari_imagine):
                    x = random.randint(0, latime - dim_fereastra[0])
                    y = random.randint(0, inaltime - dim_fereastra[1])

                # Decupează și salvează regiunea negativă
                patch_negativ = imagine[y:y+dim_fereastra[1], x:x+dim_fereastra[0]]
                nume_fisier_neg = f"{nume_imagine.split('.')[0]}_neg_{x}_{y}.jpg"
                cv2.imwrite(os.path.join(director_output, nume_fisier_neg), patch_negativ)
                print(f"Salvat negativ: {nume_fisier_neg}")

# Exemplu de utilizare
director_input = "antrenare/dad/"  # Înlocuiește cu calea către imaginile tale
fisiere_adnotari = [
    "antrenare/dad_annotations.txt",
    "antrenare/mom_annotations.txt",
    "antrenare/dexter_annotations.txt",
    "antrenare/deedee_annotations.txt",
]  # Căile către fișierele de adnotări
director_output = "poze_negative"  # Calea către directorul de output

genereaza_poze_negative(director_input, fisiere_adnotari, director_output, dim_fereastra=(64, 64), nr_neg=10)


Salvat negativ: 0001_neg_128_252.jpg
Salvat negativ: 0001_neg_102_37.jpg
Salvat negativ: 0001_neg_123_24.jpg
Salvat negativ: 0001_neg_122_257.jpg
Salvat negativ: 0001_neg_415_296.jpg
Salvat negativ: 0001_neg_174_5.jpg
Salvat negativ: 0001_neg_32_174.jpg
Salvat negativ: 0001_neg_76_284.jpg
Salvat negativ: 0001_neg_231_287.jpg
Salvat negativ: 0001_neg_133_7.jpg
Salvat negativ: 0002_neg_320_294.jpg
Salvat negativ: 0002_neg_268_241.jpg
Salvat negativ: 0002_neg_175_223.jpg
Salvat negativ: 0002_neg_404_65.jpg
Salvat negativ: 0002_neg_370_261.jpg
Salvat negativ: 0002_neg_252_257.jpg
Salvat negativ: 0002_neg_123_286.jpg
Salvat negativ: 0002_neg_23_259.jpg
Salvat negativ: 0002_neg_53_174.jpg
Salvat negativ: 0002_neg_391_233.jpg
Salvat negativ: 0003_neg_179_61.jpg
Salvat negativ: 0003_neg_3_224.jpg
Salvat negativ: 0003_neg_46_200.jpg
Salvat negativ: 0003_neg_253_15.jpg
Salvat negativ: 0003_neg_398_241.jpg
Salvat negativ: 0003_neg_279_126.jpg
Salvat negativ: 0003_neg_60_222.jpg
Salvat negativ: 00

KeyboardInterrupt: 

In [31]:
import os
import cv2

# Setări directoare
input_dir = "antrenare/dexter/"  # Directorul cu imaginile originale
annotation_file = "antrenare/dexter_annotations.txt"  # Fișierul cu adnotări
output_dir = "teste_poz_pe_formw"  # Directorul pentru salvarea fețelor

# Creează directoare de ieșire
for category in ["square", "vertical", "horizontal"]:
    os.makedirs(os.path.join(output_dir, category), exist_ok=True)

# Funcție pentru clasificarea formei dreptunghiului
def clasifica_forma(xmin, ymin, xmax, ymax):
    latime = xmax - xmin
    inaltime = ymax - ymin
    raport = latime / inaltime
    if 0.8 <= raport <= 1.2:
        return "square"
    elif raport < 0.8:
        return "vertical"
    else:
        return "horizontal"

# Citește adnotările și procesează imaginile
with open(annotation_file, "r") as f:
    for linie in f:
        nume_imagine, xmin, ymin, xmax, ymax, _ = linie.strip().split()
        xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])
        imagine_path = os.path.join(input_dir, nume_imagine)

        # Încarcă imaginea
        imagine = cv2.imread(imagine_path)
        if imagine is None:
            print(f"Nu am găsit imaginea {nume_imagine}, o sar.")
            continue

        # Decupează regiunea
        decupaj = imagine[ymin:ymax, xmin:xmax]

        # Clasifică forma
        forma = clasifica_forma(xmin, ymin, xmax, ymax)

        # Salvează imaginea decupată
        nume_nou = f"{os.path.splitext(nume_imagine)[0]}_{xmin}_{ymin}.jpg"
        cale_salveaza = os.path.join(output_dir, forma, nume_nou)
        cv2.imwrite(cale_salveaza, decupaj)
        print(f"Salvat {cale_salveaza}")


Salvat teste_poz_pe_formw\horizontal\0001_191_196.jpg
Salvat teste_poz_pe_formw\horizontal\0001_77_176.jpg
Salvat teste_poz_pe_formw\vertical\0001_293_153.jpg
Salvat teste_poz_pe_formw\horizontal\0001_362_190.jpg
Salvat teste_poz_pe_formw\square\0002_192_159.jpg
Salvat teste_poz_pe_formw\horizontal\0003_41_125.jpg
Salvat teste_poz_pe_formw\horizontal\0003_341_190.jpg
Salvat teste_poz_pe_formw\horizontal\0004_151_89.jpg
Salvat teste_poz_pe_formw\horizontal\0005_185_179.jpg
Salvat teste_poz_pe_formw\horizontal\0006_10_93.jpg
Salvat teste_poz_pe_formw\horizontal\0007_231_149.jpg
Salvat teste_poz_pe_formw\square\0008_182_101.jpg
Salvat teste_poz_pe_formw\horizontal\0009_187_129.jpg
Salvat teste_poz_pe_formw\vertical\0009_3_65.jpg
Salvat teste_poz_pe_formw\horizontal\0010_82_55.jpg
Salvat teste_poz_pe_formw\horizontal\0011_175_123.jpg
Salvat teste_poz_pe_formw\horizontal\0012_70_239.jpg
Salvat teste_poz_pe_formw\horizontal\0012_228_239.jpg
Salvat teste_poz_pe_formw\horizontal\0013_99_124.jp

In [46]:
import os
import cv2

# Setări directoare
input_dir = "output_negatives"  # Directorul care conține imaginile clasificate
output_dir = "output_negatives_redim_bun"  # Directorul pentru salvarea imaginilor redimensionate

# Dimensiuni țintă pentru fiecare categorie
dimensiuni = {
    "square": (50, 50),
    "vertical": (50, 75),
    "horizontal": (75, 50)
}

# Creează directoare pentru redimensionare
for category in dimensiuni.keys():
    os.makedirs(os.path.join(output_dir, category), exist_ok=True)

# Redimensionează imaginile
for category, dim in dimensiuni.items():
    input_path = os.path.join(input_dir, category)
    output_path = os.path.join(output_dir, category)

    # Parcurge imaginile din categoria curentă
    for img_name in os.listdir(input_path):
        img_path = os.path.join(input_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Nu am putut încărca {img_path}, o sar.")
            continue

        # Redimensionare
        img_resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

        # Salvare
        save_path = os.path.join(output_path, img_name)
        cv2.imwrite(save_path, img_resized)
        print(f"Salvat {save_path}")


Salvat output_negatives_redim\square\dad-0001_183_250.jpg
Salvat output_negatives_redim\square\dad-0001_369_283.jpg
Salvat output_negatives_redim\square\dad-0001_395_280.jpg
Salvat output_negatives_redim\square\dad-0002_271_220.jpg
Salvat output_negatives_redim\square\dad-0002_51_283.jpg
Salvat output_negatives_redim\square\dad-0004_18_70.jpg
Salvat output_negatives_redim\square\dad-0004_60_178.jpg
Salvat output_negatives_redim\square\dad-0004_72_36.jpg
Salvat output_negatives_redim\square\dad-0004_74_1.jpg
Salvat output_negatives_redim\square\dad-0005_129_273.jpg
Salvat output_negatives_redim\square\dad-0005_177_265.jpg
Salvat output_negatives_redim\square\dad-0005_189_284.jpg
Salvat output_negatives_redim\square\dad-0005_192_291.jpg
Salvat output_negatives_redim\square\dad-0005_202_98.jpg
Salvat output_negatives_redim\square\dad-0006_113_2.jpg
Salvat output_negatives_redim\square\dad-0006_308_179.jpg
Salvat output_negatives_redim\square\dad-0006_349_110.jpg
Salvat output_negatives_re

In [45]:
import os
import cv2
import random

# Setări directoare
input_dir = "antrenare/dad"  # Directorul cu imaginile originale
annotation_file = "antrenare/dad_annotations.txt"  # Fișierul cu adnotări
output_dir = "output_negatives"  # Directorul pentru imaginile negative

# Dimensiuni pentru categorii
dimensiuni = {
    "square": (50, 50),
    "vertical": (50, 75),
    "horizontal": (75, 50)
}

# Creează directoarele pentru imaginile negative
for category in dimensiuni.keys():
    os.makedirs(os.path.join(output_dir, category), exist_ok=True)

# Funcție pentru încărcarea adnotărilor
def incarca_adnotari(annotation_file):
    adnotari = {}
    if not os.path.exists(annotation_file):
        print(f"Fișierul de adnotări {annotation_file} nu există.")
        return adnotari
    with open(annotation_file, "r") as f:
        for linie in f:
            nume_imagine, xmin, ymin, xmax, ymax, _ = linie.strip().split()
            xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])
            if nume_imagine not in adnotari:
                adnotari[nume_imagine] = []
            adnotari[nume_imagine].append((xmin, ymin, xmax, ymax))
    return adnotari

# Verifică dacă un dreptunghi se suprapune cu o față existentă
def se_suprapune(rect, adnotari):
    x1, y1, x2, y2 = rect
    for (xmin, ymin, xmax, ymax) in adnotari:
        if not (x2 < xmin or x1 > xmax or y2 < ymin or y1 > ymax):
            return True
    return False

# Încărcare adnotări
adnotari = incarca_adnotari(annotation_file)

# Procesare imagini
for imagine_nume in os.listdir(input_dir):
    imagine_cale = os.path.join(input_dir, imagine_nume)

    # Ignoră fișierele care nu sunt imagini
    if not imagine_nume.endswith((".jpg", ".png")):
        continue

    imagine = cv2.imread(imagine_cale)
    if imagine is None:
        print(f"Nu am putut încărca {imagine_cale}, o sar.")
        continue

    inaltime, latime = imagine.shape[:2]
    adnotari_imagine = adnotari.get(imagine_nume, [])

    numar_decupaje = 0
    incercari_max = 100  # Număr maxim de încercări pentru a găsi decupaje valide

    while numar_decupaje < 6 and incercari_max > 0:
        # Alege o categorie aleatoare și dimensiunile asociate
        categorie = random.choice(list(dimensiuni.keys()))
        w, h = dimensiuni[categorie]

        # Generează coordonate aleatoare
        x1 = random.randint(0, latime - w)
        y1 = random.randint(0, inaltime - h)
        x2 = x1 + w
        y2 = y1 + h

        # Verifică dacă dreptunghiul nu se suprapune cu vreo față
        if not se_suprapune((x1, y1, x2, y2), adnotari_imagine):
            # Decupează și salvează
            decupaj = imagine[y1:y2, x1:x2]
            nume_salveaza = f"1dad-{os.path.splitext(imagine_nume)[0]}_{x1}_{y1}.jpg"
            cale_salveaza = os.path.join(output_dir, categorie, nume_salveaza)
            cv2.imwrite(cale_salveaza, decupaj)
            print(f"Salvat {cale_salveaza}")
            numar_decupaje += 1
        incercari_max -= 1  # Scade numărul de încercări

    # Verificare dacă nu s-au generat suficiente cadrane
    if numar_decupaje < 6:
        print(f"Nu s-au putut genera toate cadranele pentru {imagine_nume}. S-au generat {numar_decupaje}.")


Salvat output_negatives\square\dad-0001_395_280.jpg
Salvat output_negatives\horizontal\dad-0001_178_293.jpg
Salvat output_negatives\square\dad-0001_183_250.jpg
Salvat output_negatives\square\dad-0001_369_283.jpg
Salvat output_negatives\vertical\dad-0001_52_4.jpg
Salvat output_negatives\vertical\dad-0001_95_237.jpg
Salvat output_negatives\square\dad-0002_51_283.jpg
Salvat output_negatives\vertical\dad-0002_282_71.jpg
Salvat output_negatives\vertical\dad-0002_320_257.jpg
Salvat output_negatives\square\dad-0002_271_220.jpg
Salvat output_negatives\vertical\dad-0002_135_233.jpg
Salvat output_negatives\vertical\dad-0002_43_183.jpg
Salvat output_negatives\vertical\dad-0003_392_225.jpg
Salvat output_negatives\vertical\dad-0003_165_67.jpg
Salvat output_negatives\horizontal\dad-0003_83_39.jpg
Salvat output_negatives\vertical\dad-0003_62_181.jpg
Salvat output_negatives\horizontal\dad-0003_3_61.jpg
Salvat output_negatives\horizontal\dad-0003_156_97.jpg
Salvat output_negatives\square\dad-0004_72_36